In [ ]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [ ]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    start_urls = ['http://www.imdb.com/chart/top',]
   
    def parse(self, response):
        # получение таблицы со строками, хараактеризующими ТОП-фильмы
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr'
        
        )
        
        for row in table_rows:
            # для каждой строки извлечем необходимую информацию
            yield {
                # пока парсим 3 колонки
                "title": row.xpath("./td[@class='titleColumn']/a/text()").extract_first(),
                "year": row.xpath("./td[@class='titleColumn']/span/text()").extract_first().strip("() "),
                "rating": row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            }

In [ ]:
DEBUG_URL = 'http://www.imdb.com/chart/top'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)


In [ ]:
selector.xpath('.//*[@class="chart full-width" and @data-caller-name="chart-top250movie"]'
               '/tbody[@class="lister-list"]')[0].extract()

In [ ]:
selector.xpath('.//*[@class="chart full-width" and @data-caller-name="chart-top250movie"]'
               '/tbody[@class="lister-list"]/tr')[0].extract()

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

## Пример с обходом страниц

In [ ]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [ ]:
# dataclass

class MovieItem(scrapy.Item):
    title = scrapy.Field()
    rating = scrapy.Field()
    summary = scrapy.Field()
    genre = scrapy.Field()
    runtime = scrapy.Field()
    directors = scrapy.Field()
    writers = scrapy.Field()
    cast = scrapy.Field()

class ActorItem(scrapy.Item):
    actor_name = scrapy.Field()
    character = scrapy.Field()

In [ ]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    base_url = "https://imdb.com"
    start_urls = ['https://www.imdb.com/chart/top',]
   
    def parse(self, response):
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr'
        
        )
        for row in table_rows:
            rating = row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            rel_url = row.xpath("td[@class='titleColumn']/a/@href").extract_first().strip()
            row_url = self.base_url + rel_url
            yield scrapy.Request(row_url, callback=self.parseOneMovie, meta={'rating' : rating})
    
    def parseOneMovie(self, response):
        item = MovieItem()
        item["rating"] = response.meta["rating"][0]
        item['title'] = response.xpath('.//*/div[@class="title_wrapper"]/h1/text()').extract_first().strip()
        item["summary"] = response.xpath(".//*/div[@class='summary_text']/text()").extract_first().strip()
        item['directors'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract_first().strip()
        item['writers'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract()[1].strip()

        item["cast"] = list()

        for cast in response.xpath(".//table[@class='cast_list']/tr")[1:]:
            actor = ActorItem()
            
            actor["actor_name"] = cast.xpath("./td[2]/a/text()").extract_first().strip()
            actor["character"] = cast.xpath("./td[@class='character']/a/text()").extract_first()
            
            item["cast"].append(actor)

        return item

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

In [ ]:
DEBUG_URL = 'http://www.imdb.com/title/tt0019254/'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)


In [ ]:
selector.xpath(".//table[@class='cast_list']/tr")